#Introduce

This notebook is created for the case using to load the Dataset (DIV2k) with difference key, and preprocessing image:


1.   Load the dataset with key.
2.   Preprocessing image.
3.   Seperate the dataset.



In [ ]:
print('import libary...')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
  

import libary...


In [ ]:
def load_dataset(key='div2k',include_validate=True):
  print('using tensorflow_datasets to load the dataset','\nloading dataset...')
  if include_validate:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )
  else:
    (train_ds), metadata = tfds.load(
    key,
    split=['train'],
    with_info=True,
    as_supervised=True,
    )
  print(metadata)
  return (train_ds,val_ds),metadata

In [ ]:
def normalize_img(image):
  print('Normalizes images: `uint8` -> `float32`')
  return tf.cast(image, tf.float32) /255

In [ ]:
def configure_performan(ds,BUFFER_SIZE=1000,BATCH_SIZE=16):
  ds=ds.cache()
  ds=ds.shuffle(buffer_size=BUFFER_SIZE)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [ ]:
def show(image, label):
  plt.figure(figsize=(len(image.numpy())/80,len(image.numpy()[0])/80))
  plt.imshow(image)
  plt.title(np.array(label))
  plt.axis('on')

In [ ]:
def visualize_augmented(original, augmented,type='lr'):
  # plt.figure(figsize=(len(original.numpy())/80,len(original.numpy()[0])/80))
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image'+type)
  plt.imshow(original)
  plt.subplot(1,2,2)
  plt.title('Augmented image'+type)
  plt.imshow(augmented)

In [ ]:
def augment(lr,hr,with_adjust=True):
  #get the same seed to get the same operator to both lr and hr for image when we use randomly
  generator= tf.random.get_global_generator()
  seed= generator.make_seeds(2)[0]
  #mean that change the pixel value by adujst it's factor
  if with_adjust:
    #low resolution augmentation
    lr=tf.image.stateless_random_contrast(lr,0.8,1.2,seed)
    lr=tf.image.stateless_random_brightness(lr,0.10,seed)
    lr=tf.image.stateless_random_saturation(lr,0.8,1.3,seed)
    #high resolution augmentation
    hr=tf.image.stateless_random_contrast(hr,0.8,1.2,seed)
    hr=tf.image.stateless_random_brightness(hr,0.10,seed)
    hr=tf.image.stateless_random_saturation(hr,0.8,1.3,seed)
  #random rotate and flip
  roate_factor=tf.squeeze(generator.uniform([1],minval=1,maxval=5,dtype=tf.dtypes.int32), axis=0)
  lr=tf.image.rot90(lr,roate_factor)
  lr=tf.image.stateless_random_flip_left_right(lr,seed)
  hr=tf.image.rot90(hr,roate_factor)
  hr=tf.image.stateless_random_flip_left_right(hr,seed)
  return lr,hr

In [ ]:
def random_crop(lr_img, hr_img, hr_crop_size, scale):
    lr_crop_size = hr_crop_size // scale
    lr_shape = tf.shape(lr_img)[:2]
    lr_top = tf.random.uniform(shape=(), maxval=lr_shape[0] - lr_crop_size + 1, dtype=tf.int32)
    lr_left = tf.random.uniform(shape=(), maxval=lr_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    hr_top = lr_top * scale
    hr_left = lr_left * scale
    lr_crop = lr_img[lr_top:lr_top + lr_crop_size, lr_left:lr_left + lr_crop_size]
    hr_crop = hr_img[hr_top:hr_top + hr_crop_size, hr_left:hr_left + hr_crop_size]
    return lr_crop, hr_crop

In [ ]:
# Normalizes RGB images to [-1, 1].
def normalize_m11(x):
    return x / 127.5 - 1
# Normalizes RGB images to [0, 1].
def normalize_01(x):
    return x / 255.0
# Denomalizes RGB images from [-1, 1].
def denormalize_m11(x):
    return (x + 1) * 127.5